In [51]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import time
from bs4 import BeautifulSoup
import pandas as pd

In [52]:
os.environ['PATH'] += r'C:\Users\gavvi\ChromeDrivers\chrome-win64\chrome-win64'

In [53]:
driver = webdriver.Chrome()
url = "https://skb-insilico.com/dlip/compound-search/ppi-library/rule-based"
driver.get(url)

driver.implicitly_wait(5)

In [54]:
# Click on the "All" button

all_btn = driver.find_element(By.XPATH, "//button[text()=' All']")
all_btn.click()

driver.implicitly_wait(2)

In [55]:
search_btn = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CLASS_NAME, 'btn-green'))
)
search_btn.click()

In [57]:
# Extract the HTML content of the table
table_html = table.get_attribute('outerHTML')

# Use BeautifulSoup to parse the HTML
soup = BeautifulSoup(table_html, 'html.parser')

# Extract column headers
header = [th.text for th in soup.find_all('th')]

# Extract table data manually
data = []
for row in soup.find_all('tr')[1:]:  # Skip the first row as it contains headers
    row_data = [td.text for td in row.find_all('td')]
    data.append(row_data)

# Construct DataFrame
df = pd.DataFrame(data, columns=header)

# Loop through pages
page_number = 1
while True:
    try:
        # Wait for the loading overlay to disappear
        WebDriverWait(driver, 20).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, "loadingoverlay"))
        )
        
        # Find the "Next" button
        next_button = driver.find_element(By.XPATH, '//*[@id="compound-list-table_next"]/a')
        
        # Click on the "Next" button
        next_button.click()

        # Wait for the table to be present on the next page
        time.sleep(3)  # Adjust this sleep time as needed
        table = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "dataTables_scrollBody"))
        )

        # Extract the HTML content of the table
        table_html = table.get_attribute('outerHTML')

        # Use BeautifulSoup to parse the HTML
        soup = BeautifulSoup(table_html, 'html.parser')

        # Extract table data manually
        for row in soup.find_all('tr')[1:]:
            row_data = [td.text for td in row.find_all('td')]
            data.append(row_data)

        # Increment the page number
        page_number += 1

        # Check if there are more pages
        if 'disabled' in next_button.get_attribute('class') or page_number > 609:
            break

    except StaleElementReferenceException:
        continue

# Construct the final DataFrame
df = pd.DataFrame(data, columns=header)

# Display the DataFrame
print(df)

      DLiP-ID Mol Image       MW  XLogP HBA HBD     PSA nRotatableBonds nRings
0      D00000            499.449   3.49   4   1    50.8               4      5
1      D00001            490.579   4.05   5   0   67.67               6      5
2      D00002             505.08  3.868   5   2    84.5               7      4
3      D00003            450.583  2.666   5   3    98.9               7      4
4      D00004            451.567  2.902   5   3    90.9               7      4
...       ...       ...      ...    ...  ..  ..     ...             ...    ...
15223  D00BQH            598.748    4.2   7   3  125.55              11      5
15224  D00BQI             643.76  3.907   7   2   113.1              11      6
15225  D00BQJ            620.238  6.435   4   4  102.57               8      6
15226  D00BQK             625.77  5.002   7   2   113.1              10      6
15227  D00BQL            645.751  5.476   7   2   97.72              10      6

[15228 rows x 9 columns]


In [58]:
# Save the DataFrame to a CSV file
csv_file_path = "DLiP_AD.csv"
df.to_csv(csv_file_path, index=False)